In [5]:
using Random
using NNlib: conv
using Flux
Random.seed!(1234);

function conv_nowe(I, K)
    H, W, C, N = size(I) 
    HH, WW, C, F = size(K)
    H_R = 1 + H - HH
    W_R = 1 + W - WW
    out = zeros(H_R, W_R, F, N)
    for n=1:N
        for depth=1:F
            @views for r=1:H_R
                for c=1:W_R
                    out[r, c, depth, n] = sum(I[r:r+HH-1, c:c+WW-1, :, n] .* K[:, :, :, depth]) 
                end
            end
        end
    end
    return out
end




function conv_noweP(I, K)
    #I = reshape(I, 28 , 28, 1, 1)
    H, W, C, N = size(I) 
    HH, WW, C, F = size(K)
    H_R = 1 + H - HH
    W_R = 1 + W - WW
    out = zeros(H_R, W_R, F, N)
    for r=1:H_R
        @views for c=1:W_R
            r_field = I[r:r+HH-1, c:c+WW-1, :, :]
            r_field_flat = reshape(r_field, HH*WW*C, N)
            K_flat = reshape(K, HH*WW*C, F)
            out[r, c, :, :] = sum(K_flat .* r_field_flat, dims = 1)
        end
    end
    return out
end

function create_kernels_nowe(kernel_height, kernel_width, n_input, n_output)
    # Inicjalizacja Xaviera
    squid = sqrt(6 / (n_input + n_output * (kernel_width * kernel_height)))
    random_vals = randn(kernel_height, kernel_width, n_input, n_output) * squid
    return random_vals
end

function create_kernels(n_input, n_output, kernel_width, kernel_height)
    # Inicjalizacja Xaviera
    squid = sqrt(6 / (n_input + n_output * (kernel_width * kernel_height)))
    random_vals = randn(n_output, n_input, kernel_width, kernel_height) * squid
    return random_vals
end


create_kernels (generic function with 1 method)

In [6]:
test_n = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.21568628 0.53333336 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.6745098 0.99215686 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.07058824 0.8862745 0.99215686 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.19215687 0.07058824 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.67058825 0.99215686 0.99215686 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.11764706 0.93333334 0.85882354 0.3137255 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.09019608 0.85882354 0.99215686 0.83137256 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.14117648 0.99215686 0.99215686 0.6117647 0.05490196 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.25882354 0.99215686 0.99215686 0.5294118 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.36862746 0.99215686 0.99215686 0.41960785 0.003921569 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.09411765 0.8352941 0.99215686 0.99215686 0.5176471 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.6039216 0.99215686 0.99215686 0.99215686 0.6039216 0.54509807 0.043137256 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.44705883 0.99215686 0.99215686 0.95686275 0.0627451 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.011764706 0.6666667 0.99215686 0.99215686 0.99215686 0.99215686 0.99215686 0.74509805 0.13725491 0.0 0.0 0.0 0.0 0.0 0.15294118 0.8666667 0.99215686 0.99215686 0.52156866 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.07058824 0.99215686 0.99215686 0.99215686 0.8039216 0.3529412 0.74509805 0.99215686 0.94509804 0.31764707 0.0 0.0 0.0 0.0 0.5803922 0.99215686 0.99215686 0.7647059 0.043137256 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.07058824 0.99215686 0.99215686 0.7764706 0.043137256 0.0 0.007843138 0.27450982 0.88235295 0.9411765 0.1764706 0.0 0.0 0.18039216 0.8980392 0.99215686 0.99215686 0.3137255 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.07058824 0.99215686 0.99215686 0.7137255 0.0 0.0 0.0 0.0 0.627451 0.99215686 0.7294118 0.0627451 0.0 0.50980395 0.99215686 0.99215686 0.7764706 0.03529412 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.49411765 0.99215686 0.99215686 0.96862745 0.16862746 0.0 0.0 0.0 0.42352942 0.99215686 0.99215686 0.3647059 0.0 0.7176471 0.99215686 0.99215686 0.31764707 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.53333336 0.99215686 0.9843137 0.94509804 0.6039216 0.0 0.0 0.0 0.003921569 0.46666667 0.99215686 0.9882353 0.9764706 0.99215686 0.99215686 0.7882353 0.007843138 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.6862745 0.88235295 0.3647059 0.0 0.0 0.0 0.0 0.0 0.0 0.09803922 0.5882353 0.99215686 0.99215686 0.99215686 0.98039216 0.30588236 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.101960786 0.6745098 0.32156864 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.105882354 0.73333335 0.9764706 0.8117647 0.7137255 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.6509804 0.99215686 0.32156864 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2509804 0.007843138 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.9490196 0.21960784 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.96862745 0.7647059 0.15294118 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.49803922 0.2509804 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;;;];
jj = create_kernels_nowe(3, 3, 1, 6);

In [7]:
result = conv(test_n, jj);

In [8]:
import NNlib: conv, maxpool
using BenchmarkTools

In [11]:
function conv_nowe(I, K)
    H, W, C, N = size(I) 
    HH, WW, C, F = size(K)
    H_R = 1 + H - HH
    W_R = 1 + W - WW
    out = zeros(H_R, W_R, F, N)
    for n=1:N
        for depth=1:F
            @views for r=1:H_R
                for c=1:W_R
                    out[r, c, depth, n] = sum(I[r:r+HH-1, c:c+WW-1, :, n] .* K[:, :, :, depth]) 
                end
            end
        end
    end
    return out
end


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  319.400 μs …   2.315 ms  ┊ GC (min … max): 0.00% … 81.83%
 Time  (median):     335.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   367.347 μs ± 156.393 μs  ┊ GC (mean ± σ):  4.25% ±  8.28%

  █▆▄▃▃▃▃▂▁                                                     ▁
  ██████████▇▇▆▆▆▆▆▆▅▅▄▄▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▆ █
  319 μs        Histogram: log(frequency) by time       1.52 ms <

 Memory estimate: 528.23 KiB, allocs estimate: 3383.

In [10]:
@benchmark conv(test_n, jj)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   8.200 μs …   4.212 ms  ┊ GC (min … max):  0.00% … 98.09%
 Time  (median):     26.600 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   31.362 μs ± 139.288 μs  ┊ GC (mean ± σ):  17.81% ±  4.14%

  ▁▄█▆▃▂▂▂▁▁    ▁▅▇▇▅▅▆█▇▅▃▂▂▃▂▁                               ▂
  ██████████▇▇▇▆███████████████████▇██▇▇▇▇▆▇█▇▇▇▇▇▆▇▆▇███▇███▇ █
  8.2 μs        Histogram: log(frequency) by time      67.5 μs <

 Memory estimate: 82.72 KiB, allocs estimate: 40.

In [16]:
function create_kernels_w(n_input, n_output, kernel_width, kernel_height)
    # Inicjalizacja Xaviera
    squid = sqrt(6 / (n_input + n_output * (kernel_width * kernel_height)))
    random_vals = randn(n_input, kernel_width, kernel_height, n_output) * squid
    return random_vals
end


create_kernels_w (generic function with 1 method)

In [17]:
j2 = create_kernels_w(1, 3, 3, 6);


In [21]:
test_2 = reshape(test_n, 1, 28, 28, 1)
@benchmark conv_2d_combined!(test_2, j2)

In [2]:
function
    fftconv(img::Matrix,
     kernel::Matrix)
     ker = zero(img)
     ker[1:3, 1:3] .= kernel
     I, K = fft(img), fft(ker)
     J = ifft(I .* K)
     return abs.(J)
    end
    

fftconv (generic function with 1 method)

In [4]:
function create_kernels2(kernel_width, kernel_height)
    # Inicjalizacja Xaviera
    squid = sqrt(6 / (kernel_width * kernel_height))
    random_vals = randn(kernel_width, kernel_height) * squid
    return random_vals
end


create_kernels2 (generic function with 1 method)

In [10]:
test_2 = reshape(test_n, 28, 28)
k2 = create_kernels2(3, 3)

3×3 Matrix{Float64}:
 -0.0080937  0.986503  -0.112035
 -0.19367    1.07137    0.0385128
 -0.191099   0.339996   0.817187

In [19]:
using FFTW
sum(fftconv(test_2, k2))

306.62923058523165

In [20]:
@benchmark fftconv(test_2, k2)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   70.000 μs …   8.370 ms  ┊ GC (min … max): 0.00% … 69.02%
 Time  (median):      99.300 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   116.590 μs ± 258.826 μs  ┊ GC (mean ± σ):  5.11% ±  2.28%

        ▁▇█▅▃                                                    
  ▁▂▂▂▂▄█████▇▆▅▅▅▅▆▆▆▇▆▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  70 μs            Histogram: frequency by time          208 μs <

 Memory estimate: 87.91 KiB, allocs estimate: 23.

In [31]:
function
    fftconv(img::Matrix,
     kernel::Matrix)
     ker = zero(img)
     ker[1:3, 1:3] .= kernel
     I, K = fft(img), fft(ker)
     J = ifft(I .* K)
     return abs.(J)
    end


function convff(I::Array{Float64, 4}, K::Array{Float64, 4})
    H, W, C, N = size(I)
    HH, WW, C, F = size(K)
    H_R = 1 + H - HH
    W_R = 1 + W - WW
    out = zeros(HH, WW, F, N)
    for n=1:N
        for depth=1:F
            println(size(I[:, :, depth, n]))
            println(size(K[:, :, n, depth]))
            out[:, :, depth, n] = fftconv(I[:, :, depth, n], K[:, :, n, depth])
        end
    end
    return out
end
println(sum(convff(test_n, jj)))

(28, 28)
(3, 3)


DimensionMismatch: DimensionMismatch: tried to assign 28×28 array to 3×3×1×1 destination

In [ ]:
c